### Starting the Game Project
- In pygame, the origin (0,0) is at the top-left corner of the screen, and coordinates increase as you go down and to the right. On a 1200 by 800 screen, the origin is at the top-left corner, and the bottom-right corner has the coordinates(1200,800).

In [16]:
import sys
import pygame
from pygame.sprite import Group

def run_game():
    # Initialize pygame, settings, and screen object.
    pygame.init() # initialize background settings that Pygame needs to work properly
    ai_settings = Settings() # create an instance of Settings to be used throughout game
    
    # creates a display window (or surface)
    screen = pygame.display.set_mode((ai_settings.screen_width, ai_settings.screen_height)) 
    pygame.display.set_caption("Alien Invasion")
    
    # Make a ship.
    ship = Ship(ai_settings, screen)
    
    # Make a group to store bullets in.
    bullets = Group() # instance of the class pygame.sprite.Group
    
    # Make a group of aliens
    aliens = Group()
    
    # Create the fleet of aliens
    create_fleet(ai_settings, screen, ship, aliens)
    
    # Start the main loop for the game.
    while True:
        check_events(ai_settings, screen, ship, bullets)
        ship.update()
        update_bullets(bullets)
        update_screen(ai_settings, screen, ship, bullets, aliens)
        

run_game()

TypeError: 'NoneType' object cannot be interpreted as an integer

In [7]:
quit

In [11]:
# Creating a Settings Class
class Settings():
    """A class to store all settings for Alien Invasion."""
    
    def __init__(self):
        """Initialize the game's settings."""
        # Screen settings
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)
        
        # Ship settings
        self.ship_speed_factor = 1.5 # number of pixels adjusted
        
        # Bullet settings
        self.bullet_speed_factor = 1
        self.bullet_width = 3
        self.bullet_height = 15
        self.bullet_color = 60, 60, 60
        self.bullets_allowed = 3

In [12]:
# Creating the Ship Class
class Ship():
    
    def __init__(self, ai_settings, screen):
        """Initialize the ship and set its starting position."""
        self.screen = screen
        self.ai_settings = ai_settings # create attribute
        
        # Load the ship image and get its rect.
        # returns a surface representing the ship
        self.image = pygame.image.load('images/ship.bmp') 
        
        # access surface's rect attribute (rect = rectangles); allows to treat elements
        # as rectangles which are simple to deal with; one can access the top, bottom, 
        # left, and right edges, and center
        self.rect = self.image.get_rect() 
        
        # stores screen's rect
        self.screen_rect = screen.get_rect() 
        
        # Start each new ship at the bottom center of the screen.
        # places the x-coordinate of the ship's center to match the center of screen
        self.rect.centerx = self.screen_rect.centerx
        # places the y-coordinate of the ship's bottom to match the bottom of screen
        self.rect.bottom = self.screen_rect.bottom 
        
        # Store a decimal value for the ship's center.
        self.center = float(self.rect.centerx)
        
        # Movement flags
        self.moving_right = False
        self.moving_left = False
    
    def update(self):
        """Update the ship's position based on the movement flags."""
        # Update the ship's center value, not the rect.
        if self.moving_right and self.rect.right < self.screen_rect.right: # limits screen from going off screen
            self.center += self.ai_settings.ship_speed_factor
        if self.moving_left and self.rect.left > 0: # limits screen from going off left margin of screen
            self.center -= self.ai_settings.ship_speed_factor
            
        # Update rect object from self.center.
        self.rect.centerx = self.center # only integer portion of self.center will be stored in self.rect.centerx
        
    def blitme(self):
        """Draw the ship at its current location."""
        self.screen.blit(self.image, self.rect) # draws image to screen at position specified by self.rect

In [13]:
# Creating the Bullet Class
import pygame
from pygame.sprite import Sprite
# When you use sprites, you can group related elements in your 
# game and act on all the grouped elements at once.

class Bullet(Sprite):
    """A Class to manage bullets fired from the ship"""
    
    def __init__(self, ai_settings, screen, ship):
        """Create a bullet object at the ship's current position."""
        super(Bullet, self).__init__() # inherit properly from Sprite
        self.screen = screen
        
        # Create a bullet rect at (0,0) and then set correct position.
        # build bullets rect from scratch
        self.rect = pygame.Rect(0, 0, ai_settings.bullet_width, ai_settings.bullet_height) 
        # makes bullets appear at top center of ship
        self.rect.centerx = ship.rect.centerx
        self.rect.top = ship.rect.top 
        
        # Store the bullet's position as a decimal value. (the y-coordinate)
        self.y = float(self.rect.y)
        
        # Store color and speed settings
        self.color = ai_settings.bullet_color
        self.speed_factor = ai_settings.bullet_speed_factor
        
    def update(self):
        """Move the bullet up the screen."""
        # Update the decimal position of the bullet.
        self.y -= self.speed_factor
        # Update the rect position.
        self.rect.y = self.y
    
    def draw_bullet(self):
        """Draw the bullet to the screen."""
        pygame.draw.rect(self.screen, self.color, self.rect)

In [14]:
# Game Functions
        
# KEYSTROKE EVENTS
def check_keydown_events(event, ai_settings, screen, ship, bullets):
    """Respond to keypresses"""
    if event.key == pygame.K_RIGHT: # check if right arrow key is pressed
        ship.moving_right = True
    elif event.key == pygame.K_LEFT: # check if left arrow key is pressed
        ship.moving_left = True
    elif event.key == pygame.K_SPACE:
        fire_bullet(ai_settings, screen, ship, bullets)
    elif event.key == pygame.K_q:
        sys.exit()
        
            
def check_keyup_events(event, ship):
    """Respond to key releases"""
    if event.key == pygame.K_RIGHT: # check if right arrow key is pressed
        ship.moving_right = False
    elif event.key == pygame.K_LEFT: # check if left arrow key is pressed
        ship.moving_left = False

        
def check_events(ai_settings, screen, ship, bullets):
    """Respond to keypressed and mouse events."""
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()
        elif event.type == pygame.KEYDOWN: # detects a keydown event
            check_keydown_events(event, ai_settings, screen, ship, bullets)
        elif event.type == pygame.KEYUP: # detects a keyup event    
            check_keyup_events(event, ship)
        
def update_screen(ai_settings, screen, ship, bullets, aliens):
    """Update images on the screen and flip to the new screen."""
    
    # Redraw the screen during each pass through the loop.
    screen.fill(ai_settings.bg_color)
    # Redraw all bullets behind ship and aliens.
    for bullet in bullets.sprites(): # bullets.sprites() method returns a list of all sprites in the group bullets
        bullet.draw_bullet() # method from Bullet class
    # Redraw ship onscreen after filling the background
    ship.blitme()
    # Redraw alien onscreen after filling the background
    # Pygame automatically draws each element in the group at the position defined
    # by its rect attribute
    aliens.draw(screen)

    # Make the most recently drawn screen visible.
    pygame.display.flip()

def update_bullets(bullets):
    """Update position of bullets and get rid of old bullets."""
    # Update bullet positions
    bullets.update() # calls bullet.update() for each bullet we place in the group bullets
        
    # Get rid of bullets that have disappeared.
    for bullet in bullets.copy(): # you shouldn't remove items from a list or group within a for loop
        if bullet.rect.bottom <= 0:
            bullets.remove(bullet)
            
def fire_bullet(ai_settings, screen, ship, bullets):
    """Fire a bullet if limit not reached yet"""
    # Create a new bullet and add it to the bullets group.
    if len(bullets) < ai_settings.bullets_allowed: 
        new_bullet = Bullet(ai_settings, screen, ship) # creates instance of bullet
        bullets.add(new_bullet)
        
def get_number_aliens_x(ai_settings, alien_width):
    """Determine the number of aliens that fit in a row."""
    # Calculate horizontal space available for aliens
    available_space_x = ai_settings.screen_width - 2 * alien_width
    # Number of aliens that can fit in that space
    number_aliens_x = int(available_space_x / (2 * alien_width))
    return number_aliens_x

def get_number_rows(ai_settings, ship_height, alien_height):
    """Determine the number of rows of aliens that fit on the screen."""
    available_space_y = (ai_settings.screen_height - (3 * alien_height) - ship_height)
    number_rows = int(available_space_y / (2 * alien_height))

def create_alien(ai_settings, screen, aliens, alien_number, row_number):
    """Create an alien and place it in the row"""
    alien = Alien(ai_settings, screen)
    alien_width = alien.rect.width
    alien.x = alien_width + 2 * alien_width * alien_number
    alien.y = alien.rect.height + 2 * alien.rect.height * row_number
    alien.rect.x = alien.x
    aliens.add(alien)
        
def create_fleet(ai_settings, screen, ship, aliens):
    """Create a full fleet of aliens."""
    # Create an initial alien and find the number of aliens in a row.
    alien = Alien(ai_settings, screen)
    number_aliens_x = get_number_aliens_x(ai_settings, alien.rect.width)
    number_rows = get_number_rows(ai_settings, ship.rect.height, alien.rect.height)
    
    # Create the first row of aliens
    for row_number in range(number_rows):
        for alien_number in range(number_aliens_x):
            create_alien(ai_settings, screen, aliens, alien_number, row_number)
    

In [15]:
# Creating the Alien Class
import pygame
from pygame.sprite import Sprite

class Alien(Sprite):
    """A class to represent a single alien in the fleet."""
    
    def __init__(self, ai_settings, screen):
        """Initalize the alien and set its starting position."""
        super(Alien, self).__init__() # inherit properly from Sprite
        self.screen = screen
        self.ai_settings = ai_settings
        
        # Load the alien image and set its rect attribute
        self.image = pygame.image.load('images/alien.bmp')
        self.rect = self.image.get_rect()
        
        # Start each new alien near the top left of the screen.
        self.rect.x = self.rect.width
        self.rect.y = self.rect.height
        
        # Store the alien's exact position.
        self.x = float(self.rect.x)
        
    def blitme(self):
        """Draw the alien at its current location."""
        self.screen.blit(self.image, self.rect) # draws image to screen at position specified by self.rect
